In [1]:
import argparse
import glob
import json
import logging
import os
import pickle as pkl
import warnings

import hist as hist2
import numpy as np
import pandas as pd
import pyarrow
import yaml
#from systematics import get_systematic_dict, sigs
from systematicsPass import get_systematic_dict, sigs

from utils import get_common_sample_name, get_finetuned_score, get_xsecweight

logging.basicConfig(level=logging.INFO)

warnings.filterwarnings("ignore", message="Found duplicate branch ")
pd.set_option("mode.chained_assignment", None)

CATEGORY = 'pass'



def get_templates(years, channels, samples, samples_dir, regions_sel, model_path, add_fake=False):

    # add extra selections to preselection
    presel = { "mu": { "fj_mass": "fj_mass < 180",}, "ele": {"fj_mass": "fj_mass <180",}, }

        
    #mass_binning = [40,65,110,135,180]
    mass_binning =  [40,70,110,140,180]


    
    hists = hist2.Hist(
        hist2.axis.StrCategory([], name="Sample", growth=True), hist2.axis.StrCategory([], name="Systematic", growth=True), hist2.axis.StrCategory([], name="Region", growth=True),
        hist2.axis.Variable(
            #list(range(55, 255, mass_binning)),
            mass_binning, name="mass_observable", label=r"V reconstructed mass [GeV]", overflow=True,
        ), storage=hist2.storage.Weight(),
    )

    SYST_DICT = get_systematic_dict(years)
    for year in years:  # e.g. 2018, 2017, 2016APV, 2016    
        for ch in channels:  # e.g. mu, ele
            logging.info(f"Processing year {year} and {ch} channel")
            with open("../../fileset/luminosity.json") as f:
                luminosity = json.load(f)[ch][year]
            for sample in os.listdir(samples_dir[year]):
                sample_to_use = get_common_sample_name(sample)
                if sample_to_use not in samples:
                    continue
                is_data = True if sample_to_use == "Data" else False

                logging.info(f"Finding {sample} samples and should combine them under {sample_to_use}")

                out_files = f"{samples_dir[year]}/{sample}/outfiles/"
                parquet_files = glob.glob(f"{out_files}/*_{ch}.parquet")
                pkl_files = glob.glob(f"{out_files}/*.pkl")
                if not parquet_files:
                    logging.info(f"No parquet file for {sample}")
                    continue
                try:
                    data = pd.read_parquet(parquet_files)
                except pyarrow.lib.ArrowInvalid:  # empty parquet because no event passed selection
                    continue
                if len(data) == 0:
                    continue
                data["THWW"] = get_finetuned_score(data, model_path)
                data = data[data.columns.drop(list(data.filter(regex="hidNeuron")))]

                for selection in presel[ch]:
                    logging.info(f"Applying {selection} selection on {len(data)} events")
                    data = data.query(presel[ch][selection])

#*****             # get the xsecweight
                xsecweight, sumgenweights, sumpdfweights, sumscaleweights = get_xsecweight( pkl_files, year, sample, sample_to_use, is_data, luminosity)

                for region, region_sel in regions_sel.items():  # e.g. pass, fail, top control region, etc.
                #need to apply the V calibration only when making a cut on V tagger; currently the TTbar CR has no cut on tagger, so don't apply it
#**************************************************************************************
                    df = data.copy()
                    logging.info(f"Applying {region} selection on {len(df)} events")
                    df = df.query(region_sel)
                    logging.info(f"Will fill the histograms with the remaining {len(df)} events")

                    # ------------------- Nominal -------------------
                    if is_data:
                        nominal = np.ones_like(df["fj_pt"])  # for data (nominal is 1)
                    else:
                        nominal = df[f"weight_{ch}"] * xsecweight * df["weight_btag"] 
                 
                    # ------------------- QCD scale -------------------
                hists.fill( Sample=sample_to_use, Systematic="pass_nominal", Region=region, mass_observable=df["fj_mass"], weight=nominal,)
        #for variation in ["fakes_nominal", "fakes_SF_Up", "fakes_SF_Down", "fakes_DR_Up", "fakes_DR_Down"]:

    
    return hists


def fix_neg_yields(h):
    """
    Will set the bin yields of a process to 0 if the nominal yield is negative, and will
    set the yield to 0 for the full Systematic axis.
    """
    for region in h.axes["Region"]:
        for sample in h.axes["Sample"]:
            neg_bins = np.where(h[{"Sample": sample, "Systematic": "pass_nominal", "Region": region}].values() < 0)[0]

            if len(neg_bins) > 0:
                print('got neg bins')
                print(f"{region}, {sample}, has {len(neg_bins)} bins with negative yield.. will set them to 0")

                sample_index = np.argmax(np.array(h.axes["Sample"]) == sample)
                region_index = np.argmax(np.array(h.axes["Region"]) == region)

                for neg_bin in neg_bins:
                    h.view(flow=True)[sample_index, :, region_index, neg_bin + 1].value = 1e-3
                    h.view(flow=True)[sample_index, :, region_index, neg_bin + 1].variance = 1e-3



In [2]:
#years = ['2017','2018']
years = ['2016APV']

#years = ['2016', '2016APV','2017', '2018']
#years = ['2017', '2018']

outdir = 'templates'

channels = 'mu','ele'
with open("simplePass_TopCR_data.yaml", "r") as stream:
    config = yaml.safe_load(stream)

if len(years) == 4:
    save_as = "Run2"
else:
    save_as = "_".join(years)

if len(channels) == 1:
    save_as += f"_{channels[0]}_"

os.system(f"mkdir -p {outdir}")

hists = get_templates( years, channels, config["samples"], config["samples_dir"], config["regions_sel"], config["model_path"],)

fix_neg_yields(hists)

with open(f"{outdir}/hists_templates_{save_as}_data_pass_TopCR.pkl", "wb") as fp:
    print('hists', hists)
    pkl.dump(hists, fp)




INFO:root:Processing year 2016APV and mu channel
INFO:root:Finding SingleElectron_Run2016B_ver2_HIPM samples and should combine them under Data
INFO:root:Finding SingleElectron_Run2016C_HIPM samples and should combine them under Data
INFO:root:Finding SingleElectron_Run2016D_HIPM samples and should combine them under Data
INFO:root:Finding SingleElectron_Run2016E_HIPM samples and should combine them under Data
INFO:root:Finding SingleElectron_Run2016F_HIPM samples and should combine them under Data
INFO:root:Finding SingleMuon_Run2016B_ver2_HIPM samples and should combine them under Data
INFO:root:Applying fj_mass selection on 8924 events
INFO:root:Applying TopCR selection on 8197 events
INFO:root:Will fill the histograms with the remaining 275 events
INFO:root:Finding SingleMuon_Run2016C_HIPM samples and should combine them under Data
INFO:root:Applying fj_mass selection on 3932 events
INFO:root:Applying TopCR selection on 3600 events
INFO:root:Will fill the histograms with the remain

hists Hist(
  StrCategory(['Data'], growth=True, name='Sample'),
  StrCategory(['pass_nominal'], growth=True, name='Systematic'),
  StrCategory(['TopCR'], growth=True, name='Region'),
  Variable([40, 70, 110, 140, 180], name='mass_observable', label='V reconstructed mass [GeV]'),
  storage=Weight()) # Sum: WeightedSum(value=1551, variance=1551)
